In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
# pip install -q -U tensorflow-addons

In [3]:
# import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D,LocallyConnected1D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization,Add,concatenate
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [4]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [5]:
N_FEATURES = 12

In [6]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'Class_label']


In [7]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [8]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [9]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [10]:
 pd.options.display.float_format = "{:,.5f}".format

In [11]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [12]:
# import os
path='/media/naveen/nav/mat_codes/nina_DB4_codes/naveen_prep_nlw/Total_process_TT/'

In [13]:
#  for i in range(num_res_net_blocks_64):
#     x = res_net_block_64(x, 64, 3)

In [14]:
# file_path_train=path+'Train_data'+'.'+'csv'
file_path_train=path+'nina_naveen_nlw_DB4_Train'+'.'+'csv'
print(file_path_train)

/media/naveen/nav/mat_codes/nina_DB4_codes/naveen_prep_nlw/Total_process_TT/nina_naveen_nlw_DB4_Train.csv


In [15]:
file_path_test=path+'nina_naveen_nlw_DB4_Test'+'.'+'csv'
# file_path_test=path+'Test_data'+'.'+'csv'
print(file_path_test)

/media/naveen/nav/mat_codes/nina_DB4_codes/naveen_prep_nlw/Total_process_TT/nina_naveen_nlw_DB4_Test.csv


In [16]:
df_Train=read_data_Train(file_path_train)
show_basic_dataframe_info(df_Train)
df_Train.head(5)

Number of columns in the dataframe: 13
Number of rows in the dataframe: 20800000



,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,14,8,16,12,27,64,33,15,1,3,33,111,1.00000
1,21,10,15,10,28,34,27,14,1,4,12,118,1.00000
2,29,9,14,8,29,5,21,15,1,3,10,126,1.00000
3,43,1,14,6,29,7,15,20,1,4,28,127,1.00000
4,51,11,14,4,28,13,9,27,2,4,40,127,1.00000


In [17]:
df_Test=read_data_Test(file_path_test)
show_basic_dataframe_info(df_Test)
df_Test.head(5)

Number of columns in the dataframe: 13
Number of rows in the dataframe: 10400000



,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,107,14,12,7,99,19,5,71,58,102,22,91,1.00000
1,106,16,10,8,75,12,16,71,58,104,20,41,1.00000
2,102,18,9,7,48,40,40,72,54,105,16,24,1.00000
3,93,18,7,5,54,48,78,32,47,107,9,64,1.00000
4,79,19,6,3,76,29,98,4,37,107,2,83,1.00000


In [18]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,14,8,16,12,27,64,33,15,1,3,33,111,1.00000
1,21,10,15,10,28,34,27,14,1,4,12,118,1.00000
2,29,9,14,8,29,5,21,15,1,3,10,126,1.00000
3,43,1,14,6,29,7,15,20,1,4,28,127,1.00000
4,51,11,14,4,28,13,9,27,2,4,40,127,1.00000


In [19]:
scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])

In [20]:
df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])

In [21]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,-1.37008,-1.47208,-1.06140,-0.92977,-0.53847,0.45083,-0.40544,-1.11646,-1.23680,-1.61419,-0.36486,1.78064,1.00000
1,-1.18840,-1.42226,-1.08680,-0.98586,-0.51239,-0.32841,-0.56317,-1.14266,-1.23680,-1.58923,-0.88006,1.97403,1.00000
2,-0.98077,-1.44717,-1.11220,-1.04195,-0.48631,-1.08167,-0.72090,-1.11646,-1.23680,-1.61419,-0.92913,2.19505,1.00000
3,-0.61741,-1.64644,-1.11220,-1.09804,-0.48631,-1.02972,-0.87862,-0.98544,-1.23680,-1.58923,-0.48753,2.22268,1.00000
4,-0.40977,-1.39736,-1.11220,-1.15413,-0.51239,-0.87387,-1.03635,-0.80201,-1.21095,-1.58923,-0.19313,2.22268,1.00000


In [22]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,107,14,12,7,99,19,5,71,58,102,22,91,1.00000
1,106,16,10,8,75,12,16,71,58,104,20,41,1.00000
2,102,18,9,7,48,40,40,72,54,105,16,24,1.00000
3,93,18,7,5,54,48,78,32,47,107,9,64,1.00000
4,79,19,6,3,76,29,98,4,37,107,2,83,1.00000


In [23]:
df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])

In [24]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,1.04367,-1.32263,-1.16300,-1.07000,1.33949,-0.71802,-1.14150,0.35096,0.23657,0.85680,-0.63473,1.22808,1.00000
1,1.01772,-1.27281,-1.21380,-1.04195,0.71350,-0.89984,-0.85233,0.35096,0.23657,0.90672,-0.68379,-0.15332,1.00000
2,0.91390,-1.22300,-1.23920,-1.07000,0.00927,-0.17256,-0.22143,0.37717,0.13318,0.93168,-0.78193,-0.62299,1.00000
3,0.68031,-1.22300,-1.29000,-1.12609,0.16576,0.03524,0.77750,-0.67099,-0.04776,0.98160,-0.95366,0.48212,1.00000
4,0.31695,-1.19809,-1.31540,-1.18218,0.73958,-0.45828,1.30325,-1.40470,-0.30625,0.98160,-1.12539,1.00705,1.00000


In [25]:
LABEL = 'ActivityEncoded'
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
# df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())

In [26]:
# The number of steps within one time segment
TIME_PERIODS = 400
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 400

In [27]:
x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_train shape: ', x_train.shape)
# print(x_train)
print(x_train.shape[0], 'training samples')
print('y_train shape: ', y_train.shape)
# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
print('num_time_periods',num_time_periods)
print('num_sensors',num_sensors)
num_classes = le.classes_.size
print('class_list',list(le.classes_))
# input_shape = (num_time_periods,num_sensors)
# print(input_shape)
input_shape = (num_time_periods,num_sensors)
#x_train = x_train.reshape(x_train.shape[0], input_shape)
print('x_train shape:', x_train[0].shape)
print('input_shape:', input_shape)
x_train = x_train.astype('float32')
# x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
# y_train = y_train.astype('float32')
# print(y_train)
y_train_hot = np_utils.to_categorical(y_train, num_classes)
print(y_train_hot)
# y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
print('New y_train shape: ', y_train_hot.shape)

x_train shape:  (51999, 400, 12)
51999 training samples
y_train shape:  (51999,)
num_time_periods 400
num_sensors 12
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (400, 12)
input_shape: (400, 12)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (51999, 52)


In [28]:
x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_test shape: ', x_test.shape)
# print(x_train)
print(x_test.shape[0], 'testing samples')
print('y_test shape: ', y_test.shape)
# Set input_shape / reshape for Keras
#x_test = x_test.reshape(x_test.shape[0], input_shape)
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')
y_test_hot = np_utils.to_categorical(y_test, num_classes)

x_test shape:  (25999, 400, 12)
25999 testing samples
y_test shape:  (25999,)


In [29]:
# n_steps, n_length = 20, 25
# n_steps, n_length= 10, 50
# n_steps, n_length= 16, 32
n_length =  400
n_depth=12
n_channel=12
x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
print('x_train shape: ', x_train.shape)
# x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
# print('x_valid shape: ', x_valid.shape)
x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
print('x_test shape: ', x_test.shape)
n_outputs = y_train_hot.shape[1]
print('n_outputs',n_outputs)

x_train shape:  (51999, 400, 12)
x_test shape:  (25999, 400, 12)
n_outputs 52


In [30]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],400,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [31]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,12)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model        

In [32]:
if __name__ == "__main__":
    model = generate_model()

(None, 400, 12)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
(None, 400, 12, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 307200)
(None, 52)
inputs_shape (None, 400, 12)


In [38]:
verbose, epochs, batch_size = 0, 100, 32

In [39]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 30.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [40]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [41]:
adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
# sgd=tfa.optimizers.SGDW(weight_decay=0.0001,learning_rate=0.1, momentum=0.9, nesterov=False, name='SGDW')
sgd=tf.optimizers.SGD(learning_rate=1e-3, momentum=0.9, nesterov=False, name='SGD')
checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5'
# model.load_weights(checkpoint_filepath) 
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)

Model: "wei_mscnn_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main (InputLayer)               [(None, 400, 12)]    0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 400)]        0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_2 (Te [(None, 400)]        0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_4 (Te [(None, 400)]        0           main[0][0]                       
____________________________________________________________________________________

In [42]:
csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/CNN_nina_20X10.csv', append=True, separator=';')
history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)

Epoch 1/100
1625/1625 [==============================] - ETA: 0s - loss: 3.5750 - accuracy: 0.1196
Epoch 00001: val_accuracy improved from -inf to 0.14224, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
1625/1625 [==============================] - 619s 381ms/step - loss: 3.5750 - accuracy: 0.1196 - val_loss: 3.4566 - val_accuracy: 0.1422 - lr: 0.0010
Epoch 2/100
1625/1625 [==============================] - ETA: 0s - loss: 3.5279 - accuracy: 0.1331
Epoch 00002: val_accuracy improved from 0.14224 to 0.16712, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
1625/1625 [==============================] - 554s 341ms/step - loss: 3.5279 - accuracy: 0.1331 - val_loss: 3.3585 - val_accuracy: 0.1671 - lr: 0.0010
Epoch 3/100
1625/1625 [==============================] - ETA: 0s - loss: 3.4655 - accuracy: 0.1455
Epoch 00003: val_accuracy improved from 0.16712 to 0.17181, saving model t

1625/1625 [==============================] - ETA: 0s - loss: 2.8855 - accuracy: 0.2471
Epoch 00022: val_accuracy improved from 0.29451 to 0.29690, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
1625/1625 [==============================] - 543s 334ms/step - loss: 2.8855 - accuracy: 0.2471 - val_loss: 2.7115 - val_accuracy: 0.2969 - lr: 0.0010
Epoch 23/100
1625/1625 [==============================] - ETA: 0s - loss: 2.8709 - accuracy: 0.2504
Epoch 00023: val_accuracy did not improve from 0.29690
1625/1625 [==============================] - 537s 330ms/step - loss: 2.8709 - accuracy: 0.2504 - val_loss: 2.7098 - val_accuracy: 0.2924 - lr: 0.0010
Epoch 24/100
1625/1625 [==============================] - ETA: 0s - loss: 2.8634 - accuracy: 0.2532
Epoch 00024: val_accuracy improved from 0.29690 to 0.29851, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
1625/1625 [===============

Epoch 44/100
1625/1625 [==============================] - ETA: 0s - loss: 2.5203 - accuracy: 0.3092
Epoch 00044: val_accuracy did not improve from 0.33278
1625/1625 [==============================] - 535s 329ms/step - loss: 2.5203 - accuracy: 0.3092 - val_loss: 2.4620 - val_accuracy: 0.3304 - lr: 1.0000e-04
Epoch 45/100
1625/1625 [==============================] - ETA: 0s - loss: 2.5073 - accuracy: 0.3132
Epoch 00045: val_accuracy improved from 0.33278 to 0.33294, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
1625/1625 [==============================] - 542s 334ms/step - loss: 2.5073 - accuracy: 0.3132 - val_loss: 2.4510 - val_accuracy: 0.3329 - lr: 1.0000e-04
Epoch 46/100
1625/1625 [==============================] - ETA: 0s - loss: 2.4982 - accuracy: 0.3139
Epoch 00046: val_accuracy improved from 0.33294 to 0.33351, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB4_20X10/checkpoint.hdf5
1625/

Epoch 68/100
1625/1625 [==============================] - ETA: 0s - loss: 2.4171 - accuracy: 0.3311
Epoch 00068: val_accuracy did not improve from 0.33863
1625/1625 [==============================] - 536s 330ms/step - loss: 2.4171 - accuracy: 0.3311 - val_loss: 2.4189 - val_accuracy: 0.3374 - lr: 1.0000e-05
Epoch 69/100
1625/1625 [==============================] - ETA: 0s - loss: 2.4144 - accuracy: 0.3313
Epoch 00069: val_accuracy did not improve from 0.33863
1625/1625 [==============================] - 537s 330ms/step - loss: 2.4144 - accuracy: 0.3313 - val_loss: 2.4190 - val_accuracy: 0.3375 - lr: 1.0000e-05
Epoch 70/100
1625/1625 [==============================] - ETA: 0s - loss: 2.4172 - accuracy: 0.3337
Epoch 00070: val_accuracy did not improve from 0.33863
1625/1625 [==============================] - 536s 330ms/step - loss: 2.4172 - accuracy: 0.3337 - val_loss: 2.4211 - val_accuracy: 0.3364 - lr: 1.0000e-05
Epoch 71/100
1625/1625 [==============================] - ETA: 0s - loss:

1625/1625 [==============================] - ETA: 0s - loss: 2.3961 - accuracy: 0.3340
Epoch 00094: val_accuracy did not improve from 0.33990
1625/1625 [==============================] - 535s 330ms/step - loss: 2.3961 - accuracy: 0.3340 - val_loss: 2.4158 - val_accuracy: 0.3380 - lr: 1.0000e-06
Epoch 95/100
1625/1625 [==============================] - ETA: 0s - loss: 2.4040 - accuracy: 0.3335
Epoch 00095: val_accuracy did not improve from 0.33990
1625/1625 [==============================] - 535s 330ms/step - loss: 2.4040 - accuracy: 0.3335 - val_loss: 2.4218 - val_accuracy: 0.3372 - lr: 1.0000e-06
Epoch 96/100
1625/1625 [==============================] - ETA: 0s - loss: 2.3951 - accuracy: 0.3330
Epoch 00096: val_accuracy did not improve from 0.33990
1625/1625 [==============================] - 536s 330ms/step - loss: 2.3951 - accuracy: 0.3330 - val_loss: 2.4158 - val_accuracy: 0.3377 - lr: 1.0000e-06
Epoch 97/100
1625/1625 [==============================] - ETA: 0s - loss: 2.3957 - acc

In [43]:
best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
print('epoch_number',best_index+1)
print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])

epoch_number 75
train accuracy and validation accuracy 0.3354487717151642 0.33989769220352173


In [44]:
model.load_weights(checkpoint_filepath) 
_, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
print('test_accuracy',testaccuracy)
test_acc.append(testaccuracy)
print(test_acc)

813/813 [==============================] - 58s 71ms/step - loss: 2.4145 - accuracy: 0.3399
test_accuracy 0.33989769220352173
[0.33989769220352173]


In [45]:
statistics.mean(test_acc)

0.33989769220352173